# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 3 2023 9:47AM,256594,10,PRF-51778,Bio-PRF,Released
1,Feb 3 2023 9:47AM,256594,10,PRF-51790,Bio-PRF,Released
2,Feb 3 2023 9:47AM,256594,10,PRF-51832,Bio-PRF,Released
3,Feb 3 2023 9:47AM,256594,10,PRF-51837,Bio-PRF,Released
4,Feb 3 2023 9:44AM,256595,15,8990185,"Brookfield Pharmaceuticals, LLC",Released
5,Feb 3 2023 9:44AM,256595,15,8990186,"Brookfield Pharmaceuticals, LLC",Released
6,Feb 3 2023 9:44AM,256595,15,8990187,"Brookfield Pharmaceuticals, LLC",Released
7,Feb 3 2023 9:44AM,256595,15,8990188,"Brookfield Pharmaceuticals, LLC",Released
8,Feb 3 2023 9:44AM,256595,15,8990189,"Brookfield Pharmaceuticals, LLC",Released
9,Feb 3 2023 9:44AM,256595,15,8990190,"Brookfield Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,256591,Released,3
30,256592,Released,9
31,256593,Released,7
32,256594,Released,4
33,256595,Released,26


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256591,NaN,NaN,3.0
256592,NaN,NaN,9.0
256593,NaN,NaN,7.0
256594,NaN,NaN,4.0
256595,NaN,NaN,26.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256471,10.0,0.0,3.0
256487,0.0,0.0,1.0
256505,0.0,1.0,0.0
256507,0.0,0.0,1.0
256513,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256471,10,0,3
256487,0,0,1
256505,0,1,0
256507,0,0,1
256513,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256471,10,0,3
1,256487,0,0,1
2,256505,0,1,0
3,256507,0,0,1
4,256513,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256471,10,,3
1,256487,,,1
2,256505,,1,
3,256507,,,1
4,256513,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 3 2023 9:47AM,256594,10,Bio-PRF
4,Feb 3 2023 9:44AM,256595,15,"Brookfield Pharmaceuticals, LLC"
30,Feb 3 2023 9:41AM,256593,10,Bio-PRF
37,Feb 3 2023 9:39AM,256592,15,"Alliance Pharma, Inc."
46,Feb 3 2023 9:33AM,256590,10,ISDIN Corporation
114,Feb 3 2023 9:29AM,256591,10,Emerginnova
117,Feb 3 2023 9:23AM,256589,19,"AdvaGen Pharma, Ltd"
121,Feb 3 2023 9:18AM,256588,10,ISDIN Corporation
122,Feb 3 2023 9:10AM,256585,10,ISDIN Corporation
144,Feb 3 2023 9:10AM,256586,16,Sartorius Bioprocess Solutions


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 3 2023 9:47AM,256594,10,Bio-PRF,,,4
1,Feb 3 2023 9:44AM,256595,15,"Brookfield Pharmaceuticals, LLC",,,26
2,Feb 3 2023 9:41AM,256593,10,Bio-PRF,,,7
3,Feb 3 2023 9:39AM,256592,15,"Alliance Pharma, Inc.",,,9
4,Feb 3 2023 9:33AM,256590,10,ISDIN Corporation,,,68
5,Feb 3 2023 9:29AM,256591,10,Emerginnova,,,3
6,Feb 3 2023 9:23AM,256589,19,"AdvaGen Pharma, Ltd",,2,2
7,Feb 3 2023 9:18AM,256588,10,ISDIN Corporation,,,1
8,Feb 3 2023 9:10AM,256585,10,ISDIN Corporation,,,22
9,Feb 3 2023 9:10AM,256586,16,Sartorius Bioprocess Solutions,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 3 2023 9:47AM,256594,10,Bio-PRF,4,,
1,Feb 3 2023 9:44AM,256595,15,"Brookfield Pharmaceuticals, LLC",26,,
2,Feb 3 2023 9:41AM,256593,10,Bio-PRF,7,,
3,Feb 3 2023 9:39AM,256592,15,"Alliance Pharma, Inc.",9,,
4,Feb 3 2023 9:33AM,256590,10,ISDIN Corporation,68,,
5,Feb 3 2023 9:29AM,256591,10,Emerginnova,3,,
6,Feb 3 2023 9:23AM,256589,19,"AdvaGen Pharma, Ltd",2,2,
7,Feb 3 2023 9:18AM,256588,10,ISDIN Corporation,1,,
8,Feb 3 2023 9:10AM,256585,10,ISDIN Corporation,22,,
9,Feb 3 2023 9:10AM,256586,16,Sartorius Bioprocess Solutions,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 3 2023 9:47AM,256594,10,Bio-PRF,4,,
1,Feb 3 2023 9:44AM,256595,15,"Brookfield Pharmaceuticals, LLC",26,,
2,Feb 3 2023 9:41AM,256593,10,Bio-PRF,7,,
3,Feb 3 2023 9:39AM,256592,15,"Alliance Pharma, Inc.",9,,
4,Feb 3 2023 9:33AM,256590,10,ISDIN Corporation,68,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 3 2023 9:47AM,256594,10,Bio-PRF,4.0,NaN,NaN
1,Feb 3 2023 9:44AM,256595,15,"Brookfield Pharmaceuticals, LLC",26.0,NaN,NaN
2,Feb 3 2023 9:41AM,256593,10,Bio-PRF,7.0,NaN,NaN
3,Feb 3 2023 9:39AM,256592,15,"Alliance Pharma, Inc.",9.0,NaN,NaN
4,Feb 3 2023 9:33AM,256590,10,ISDIN Corporation,68.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 3 2023 9:47AM,256594,10,Bio-PRF,4.0,0.0,0.0
1,Feb 3 2023 9:44AM,256595,15,"Brookfield Pharmaceuticals, LLC",26.0,0.0,0.0
2,Feb 3 2023 9:41AM,256593,10,Bio-PRF,7.0,0.0,0.0
3,Feb 3 2023 9:39AM,256592,15,"Alliance Pharma, Inc.",9.0,0.0,0.0
4,Feb 3 2023 9:33AM,256590,10,ISDIN Corporation,68.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2309216,112.0,2.0,0.0
12,1026191,16.0,2.0,0.0
15,769658,38.0,0.0,10.0
16,513131,2.0,0.0,0.0
19,1282799,18.0,4.0,0.0
21,1282659,3.0,2.0,0.0
22,513067,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2309216,112.0,2.0,0.0
1,12,1026191,16.0,2.0,0.0
2,15,769658,38.0,0.0,10.0
3,16,513131,2.0,0.0,0.0
4,19,1282799,18.0,4.0,0.0
5,21,1282659,3.0,2.0,0.0
6,22,513067,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,112.0,2.0,0.0
1,12,16.0,2.0,0.0
2,15,38.0,0.0,10.0
3,16,2.0,0.0,0.0
4,19,18.0,4.0,0.0
5,21,3.0,2.0,0.0
6,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,112.0
1,12,Released,16.0
2,15,Released,38.0
3,16,Released,2.0
4,19,Released,18.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21,22
Status,,,,,,,
Completed,0.0,0.0,10.0,0.0,0.0,0.0,0.0
Executing,2.0,2.0,0.0,0.0,4.0,2.0,0.0
Released,112.0,16.0,38.0,2.0,18.0,3.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21,22
0,Completed,0.0,0.0,10.0,0.0,0.0,0.0,0.0
1,Executing,2.0,2.0,0.0,0.0,4.0,2.0,0.0
2,Released,112.0,16.0,38.0,2.0,18.0,3.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21,22
0,Completed,0.0,0.0,10.0,0.0,0.0,0.0,0.0
1,Executing,2.0,2.0,0.0,0.0,4.0,2.0,0.0
2,Released,112.0,16.0,38.0,2.0,18.0,3.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()